In [1]:
#Dependencies
import pandas as pd
import urllib.parse
from io import StringIO
import datetime
import calendar
import time
import os
from data_utils import establish_nys_session
import warnings
warnings.filterwarnings('ignore')

Local configuration not found or error in importing: No module named 'local_config'


Read the CSVs into a pandas df

In [2]:
folder_path = 'Ressources/'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

dataframes = []  # List to hold all dataframes

for file in files:
    file_path = os.path.join(folder_path, file)  # Full path to file
    df = pd.read_csv(file_path)  # Read the file into a DataFrame
    dataframes.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames in the list into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df

,date,station_complex,borough,payment_method,total_ridership,lat,lng
0,2023-01-01,103 St (1),Manhattan,metrocard,2385,40.799446,-73.968376
1,2023-01-01,103 St (1),Manhattan,omny,2115,40.799446,-73.968376
2,2023-01-01,103 St (6),Manhattan,metrocard,1790,40.790600,-73.947479
3,2023-01-01,103 St (6),Manhattan,omny,1043,40.790600,-73.947479
4,2023-01-01,"103 St (C,B)",Manhattan,metrocard,830,40.796093,-73.961456
...,...,...,...,...,...,...,...
671471,2024-04-05,WTC Cortlandt (1),Manhattan,omny,6646,40.711834,-74.012190
671472,2024-04-05,York St (F),Brooklyn,metrocard,3884,40.701397,-73.986750
671473,2024-04-05,York St (F),Brooklyn,omny,6378,40.701397,-73.986750
671474,2024-04-05,Zerega Av (6),Bronx,metrocard,1032,40.836487,-73.847040


What are the most popular stations per Year?

In [15]:
#add year column
combined_df['date'] = combined_df['date'].astype(str)
#df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
combined_df['year']=combined_df['date'].dt.strftime('%y')
combined_df.dtypes
#data['Month & Year Date'] = data['Date'].dt.strftime('%b') + '-' + data['Date'].dt.strftime('%y')

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
#mta_grouped_df=combined_df.grouby['']